In [ ]:
import math, os, re, warnings, random
import tensorflow as tf
import numpy as np
import pandas as pd
import librosa
import tensorflow.io
from kaggle_datasets import KaggleDatasets
import matplotlib.pyplot as plt
from IPython.display import Audio
from tensorflow.keras import Model, layers
from sklearn.model_selection import KFold
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.layers import GlobalAveragePooling2D, Input, Dense, Dropout, GaussianNoise
from tensorflow.keras.applications import ResNet50
# import efficientnet.keras as efn
import seaborn as sns

In [ ]:
traindf = pd.read_csv(r'../input/rfcx-species-audio-detection/train_tp.csv')
traindf.head()

In [ ]:
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def serialize_example(wav, recording_id, target, song_id, tmin,fmin, tmax, fmax):
    feature = {
      'wav': _bytes_feature(wav),
      'recording_id': _bytes_feature(recording_id),
      'target': _float_feature(target),
      'song_id': _float_feature(song_id),
      'tmin': _float_feature(tmin),
      'fmin' : _float_feature(fmin),
      'tmax': _float_feature(tmax),
      'fmax' : _float_feature(fmax),
    }
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString() 


In [ ]:
from sklearn.model_selection import StratifiedKFold

tfrec_num = 0
kfold = StratifiedKFold(n_splits=10, shuffle=False)
for fold, (train_idx, test_idx) in enumerate(kfold.split(traindf['recording_id'], traindf['species_id'])):
    x_train , y_train = traindf['recording_id'][test_idx] , traindf['species_id'][test_idx]
   
    with tf.io.TFRecordWriter('tp%.2i-%.2i.tfrec'%(tfrec_num, len(test_idx))) as writer:
        print('Writing_tfrecords ',fold)
        for recording_id , true_value in zip(x_train, y_train): 
            wav, _ = librosa.load(f'../input/rfcx-species-audio-detection/train/{recording_id}.flac', sr = None)
            label_info = traindf.loc[traindf['recording_id'] == str(recording_id)].values[0]
            wav = tf.audio.encode_wav(tf.reshape(wav,(wav.shape[0], 1)) ,sample_rate = 48000)
            recording_id = label_info[0].encode()
            target = label_info[1]
            song_id = label_info[2]
            tmin = label_info[3]
            fmin = label_info[4]
            tmax = label_info[5]
            fmax = label_info[6]
            example = serialize_example(wav, recording_id, target, song_id, tmin,fmin, tmax, fmax)
            writer.write(example)
    tfrec_num += 1
